<a href="https://colab.research.google.com/github/Amsterdam-Internships/Readability-Lexical-Simplification/blob/master/Fine_tune_on_Readability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 6.5 MB 32.3 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Nieuwe sectie

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, AutoModel, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn
from torch.optim import Adam

import random
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import time
import datetime
import os

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-large-cased-whole-word-masking")
model = BertForSequenceClassification.from_pretrained("bert-large-cased-whole-word-masking", num_labels = 2, output_attentions = False, output_hidden_states = False)

Some weights of the model checkpoint at bert-large-cased-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the m

In [ ]:
df_full_docs = pd.read_csv("simplification_classification.csv")
# df_full_docs.head()
# # Get the lists of documents and their labels
instances = df_full_docs['nr'].tolist()
labels = df_full_docs['level'].tolist()

j=0
for i in instances:
  print(i)
  print()
  j+=1
  if j>10:
    break
# print(instances[0:10])
# Encode the labels into numerical values
# labels = [0 if label=='easy' else 1 for label in labels]



A protocarnivorous plant is a plant that traps small animals -LRB- usually insects and arthropods -RRB- much like carnivorous plants do . Unlike carnivorous plants , these plants can not get the nutritients from the trapped prey , though .

Physicians in the Third Reich were aware that smoking is responsible for cardiac diseases , which were considered to be the most serious diseases resulting from smoking .

It should be kept away from extremely cold drafts . It grows best in strong sunlight .

He still has his money in his wallet , but his identification cards are missing .

The Balkan range runs 560 km from the Vrashka Chuka Peak on the border between Bulgaria and eastern Serbia eastward through central Bulgaria to Cape Emine on the Black Sea .

After appearing in facsimile in an article written by John Nicolay in 1894 , the Nicolay Copy was presumably among the papers passed to Hay by Nicolay 's daughter Helen upon Nicolay 's death in 1901 . Robert Lincoln began a search for the or

In [ ]:
max_len = 0
len_sum = 0

# Make lists of both instance types (easy-to-read and standard)
easy_instances = [x for (x, label) in zip(instances, labels) if label == 1]
standard_instances = [x for (x, label) in zip(instances, labels) if label == 0]

for instance in easy_instances:
    input_ids = tokenizer.encode(instance, add_special_tokens=True)
    if len(input_ids)> max_len:
      max_len = max(max_len, len(input_ids))
      print(instance)
    len_sum += len(input_ids)

print('EASY: Max instance length:', max_len)
print('EASY: Average instance length:', (len_sum / len(easy_instances)))

for instance in standard_instances:
    input_ids = tokenizer.encode(instance, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    len_sum += len(input_ids)

print('STANDARD: Max instance length:', max_len)
print('STANDARD: Average instance length:', (len_sum / len(standard_instances)))

A protocarnivorous plant is a plant that traps small animals -LRB- usually insects and arthropods -RRB- much like carnivorous plants do . Unlike carnivorous plants , these plants can not get the nutritients from the trapped prey , though .
Sarbhana is located in the west of the district , it is next to following Union Councils , to the north by Kakul , Kuthwal and Beerangali , to the east by Phalkot , to the south by Namli Maira , Bagnotar and Bagh , and to the west by Dhamtour -LRB- Sungli Narrian village -RRB- .
The table setting is called 3 cheop -LRB- ì 1\/4 ì -RRB- , 5 cheop -LRB- ì ì -RRB- , 7 cheop -LRB- ì ì -RRB- , 9 cheop -LRB- ê ì -RRB- , 12 cheop -LRB- ì ì ì -RRB- bansang . This depends on the number of banchan there is .
Some of the greatest orchestras today include : the New York Philharmonic Orchestra , the Boston Symphony Orchestra , the Cleveland Orchestra , the Los Angeles Philharmonic Orchestra , the London Symphony Orchestra , the London Philharmonic Orchestra , the 

In [ ]:
# Split data into train set (90%) and test set (10%)
train_instances, test_instances, train_labels, test_labels = train_test_split(instances, labels, test_size=.1, random_state=42, shuffle=True)
  
# Also create a validation set (25% of the train set)
train_instances, val_instances, train_labels, val_labels = train_test_split(train_instances, train_labels, test_size=.25, random_state=42)


In [ ]:
# Tokenize all of the instances and map the tokens to their word IDs.
train_input_ids = []
train_attention_masks = []

# For every training instance...
for instance in train_instances[:10000]:
    encoded_dict = tokenizer.encode_plus(
                        instance,                      # Instance to encode.
                        add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                        max_length = 155,              # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,  # Construct attn. masks.
                        return_tensors = 'pt',         # Return pytorch tensors.
                   )
    
    # Add the encoded instance to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# Print instance 0, now as a list of IDs.
print('Original: ', train_instances[0])
print('Token IDs:', train_input_ids[0])

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels[:10000])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  Ernst Mayr judged him to be the most important evolutionary thinker between Darwin and the evolutionary synthesis around 1930-40 , and was `` one of the great biologists of all time '' .
Token IDs: tensor([  101, 10624,  1318,  1197, 13680,  1140,  1106,  1129,  1103,  1211,
         1696, 14745,  1341,  1200,  1206, 11555,  1105,  1103, 14745, 11362,
         1213,  3630,   118,  1969,   117,  1105,  1108,   169,   169,  1141,
         1104,  1103,  1632, 24742,  1116,  1104,  1155,  1159,   112,   112,
          119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           

In [ ]:
val_input_ids = []
val_attention_masks = []

for instance in val_instances[:2500]:
    encoded_dict = tokenizer.encode_plus(
                        instance,                      # Sentence to encode.
                        add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                        max_length = 155,               # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,  # Construct attn. masks.
                        return_tensors = 'pt',         # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    val_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    val_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.tensor(val_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', val_instances[0])
print('Token IDs:', val_input_ids[0])

# Combine the training inputs into a TensorDataset.
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels[:2500])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  The book has sold over 1.2 million copies in the subsequent four years , appearing on The New York Times children 's Best Seller list and being listed as one of the Young Adult Library Services Association 's Best Books for Young Adults , among other awards .
Token IDs: tensor([  101,  1109,  1520,  1144,  1962,  1166,   122,   119,   123,  1550,
         4034,  1107,  1103,  4194,  1300,  1201,   117,  5452,  1113,  1109,
         1203,  1365,  2706,  1482,   112,   188,  1798, 22087,  9860,  2190,
         1105,  1217,  2345,  1112,  1141,  1104,  1103,  2701, 11635,  3371,
         4326,  1791,   112,   188,  1798,  5063,  1111,  2701, 15584,   117,
         1621,  1168,  3745,   119,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,  

In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 2

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# # I believe the 'W' stands for 'Weight Decay fix"
# optimizer = AdamW(model.parameters(),
#                   lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
#                 )

optimizer = Adam(model.parameters(), lr=1e-6)

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           zztoken_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)
        
        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  5,000.    Elapsed: 0:10:58.
